# Tour of the universe

This Jupyter notebook models a tour of the universe at constant 1g acceleration. We explore journeys of between 1 and 47 billion light years, the size of the visible universe. We calculate the time experienced by the traveller and the coordinate time (eg on Earth).

## Some example distances

To give an idea of the scale of the distances involved, here are some example distances and the time experienced by the traveller on a 1g acceleration journey, stopping halfway to turn around and decelerate.

See the [solar system notebook](https://github.com/lookbusy1344/Relativity/blob/main/Python/Solar%20system.ipynb) for shorter distances.

| Light Years   | Description                              | Tour time (years) |
|---------------|------------------------------------------|-------------------|
| 4.2           | The nearest star, Proxima Centauri       | 3.5 |
| 100,000       | Width of our galaxy                      | 22 |
| 2.5 million   | Distance to Andromeda galaxy             | 28 |
| 47 billion    | Edge of observable universe              | 48 |


In [3]:
# Light years to calculate

distances: list[int] = [
    1,
    2,
    3,
    4,
    5,  # nearest star (approx)
    10,
    50,
    100,
    200,
    500,
    1_000,
    2_000,
    5_000,
    10_000,
    20_000,
    50_000,
    100_000,  # diameter of galaxy
    500_000,
    1_000_000,
    2_500_000,  # distance to Andromeda
    5_000_000,
    10_000_000,
    25_000_000,
    1_000_000_000,
    47_000_000_000,  # distance to the edge of the observable universe
]

## Code to calculate the journey

Has option for flip-and-burn at the halfway point, to slow down and return to the start, or to continue accelerating to the destination.

In [4]:
from mpmath import mp
import relativity_lib as rl
from prettytable import PrettyTable, TableStyle
from IPython.display import Markdown

# use 100 decimal places
rl.configure(300)
table = PrettyTable(
    ["Light years", "Prop Yrs", "Co'rd Yrs", "Peak c", "Lorentz", "Apparent c"]
)
table.set_style(TableStyle.MARKDOWN)
table.align = "r"  # right align everything


def process_dist(distance_ly, flip: bool) -> None:
    accel = "1"
    distance = rl.light_year * rl.ensure(distance_ly)
    a = rl.g * mp.mpf(accel)

    if flip:
        # seconds to burn to get required distance (proper time)
        burn_sec = rl.relativistic_time_for_distance(
            a, distance / 2
        )  # accelerate half-way

        # convert to years
        burn_years = burn_sec * 2 / rl.seconds_per_year

        # coordinate time (outside observer
        coord_years = rl.coordinate_time(a, burn_sec) * 2 / rl.seconds_per_year

        # work out top speed as fraction of light speed
        peak_velocity = rl.relativistic_velocity(a, burn_sec) / rl.c

        # peak lorentz factor
        peak_lorentz = rl.lorentz_factor(peak_velocity * rl.c)
    else:
        # seconds to burn to get required distance (proper time)
        burn_sec = rl.relativistic_time_for_distance(a, distance)

        # convert to years
        burn_years = burn_sec / rl.seconds_per_year

        # coordinate time (outside observer
        coord_years = rl.coordinate_time(a, burn_sec) / rl.seconds_per_year

        # work out top speed as fraction of light speed
        peak_velocity = rl.relativistic_velocity(a, burn_sec) / rl.c

        # peak lorentz factor
        peak_lorentz = rl.lorentz_factor(peak_velocity * rl.c)

    apparent_speed = (
        rl.ensure(distance_ly) / burn_years
    )  # apparent speed in proper years per light year

    table.add_row(
        [
            rl.format_mpf(distance_ly, 0),
            rl.format_mpf(burn_years, 2),
            rl.format_mpf(coord_years, 1),
            rl.format_mpf_significant(peak_velocity, 2, "9"),
            rl.format_mpf(peak_lorentz, 1),
            rl.format_mpf(apparent_speed, 1),
        ]
    )

## Flip-and-burn results

The following table shows 'flip and burn' times for various journeys. `Apparent c` is the coordinate journey length (uncontracted) divided by proper years experienced by the traveller. Even though the traveller will always measure their speed as less than c (`peak c`), as the lorentz factor causes length contraction the 'apparent' speed can be greater than c. Eg you can travel 25 million light years in 33 years of proper time, which is an apparent speed of 756,097 * c.
This hyper-c apparent velocity is only experienced for the traveller, not for the outside universe.

Using flip-and-burn you can reach the closest stars in about 4 years, and reach the Andromeda galaxy in 28 years.

In [5]:
for distance in distances:
    process_dist(distance, True)

print("Prop years = proper years, time for the traveller")
print("Co'ord years = coordinate years, eg time from Earth")
print("Peak c = the peak velocity as a fraction of the speed of light")
markdown_table = table.get_string()
display(Markdown(markdown_table))

Prop years = proper years, time for the traveller
Co'ord years = coordinate years, eg time from Earth
Peak c = the peak velocity as a fraction of the speed of light


|    Light years | Prop Yrs |        Co'rd Yrs |                    Peak c |          Lorentz |    Apparent c |
|--------------: |--------: |----------------: |-------------------------: |----------------: |-------------: |
|              1 |     1.89 |              2.2 |                      0.75 |              1.5 |           0.5 |
|              2 |     2.58 |              3.4 |                      0.87 |              2.0 |           0.7 |
|              3 |     3.07 |              4.5 |                     0.919 |              2.5 |           0.9 |
|              4 |     3.45 |              5.6 |                     0.945 |              3.0 |           1.1 |
|              5 |     3.77 |              6.6 |                     0.960 |              3.5 |           1.3 |
|             10 |     4.85 |             11.7 |                     0.986 |              6.1 |           2.0 |
|             50 |     7.71 |             51.9 |                   0.99930 |             26.8 |           6.4 |
|            100 |     9.02 |            101.9 |                   0.99981 |             52.6 |          11.0 |
|            200 |    10.34 |            201.9 |                  0.999953 |            104.2 |          19.3 |
|            500 |    12.10 |            501.9 |                 0.9999925 |            259.0 |          41.2 |
|          1,000 |    13.44 |          1,001.9 |                 0.9999981 |            517.1 |          74.3 |
|          2,000 |    14.78 |          2,001.9 |                0.99999953 |          1,033.2 |         135.2 |
|          5,000 |    16.56 |          5,001.9 |               0.999999924 |          2,581.7 |         301.8 |
|         10,000 |    17.90 |         10,001.9 |               0.999999981 |          5,162.4 |         558.4 |
|         20,000 |    19.24 |         20,001.9 |              0.9999999953 |         10,323.9 |       1,039.0 |
|         50,000 |    21.02 |         50,001.9 |             0.99999999924 |         25,808.3 |       2,378.2 |
|        100,000 |    22.36 |        100,001.9 |             0.99999999981 |         51,615.7 |       4,470.8 |
|        500,000 |    25.48 |        500,001.9 |           0.9999999999924 |        258,074.8 |      19,619.1 |
|      1,000,000 |    26.82 |      1,000,001.9 |           0.9999999999981 |        516,148.6 |      37,274.2 |
|      2,500,000 |    28.60 |      2,500,001.9 |          0.99999999999969 |      1,290,370.0 |      87,402.1 |
|      5,000,000 |    29.94 |      5,000,001.9 |         0.999999999999924 |      2,580,739.1 |     166,965.3 |
|     10,000,000 |    31.28 |     10,000,001.9 |         0.999999999999981 |      5,161,477.3 |     319,598.4 |
|     25,000,000 |    33.06 |     25,000,001.9 |        0.9999999999999969 |     12,903,691.9 |     756,097.5 |
|  1,000,000,000 |    40.21 |  1,000,000,001.9 |     0.9999999999999999981 |    516,147,638.7 |  24,868,531.9 |
| 47,000,000,000 |    47.67 | 47,000,000,001.9 | 0.99999999999999999999915 | 24,258,938,976.5 | 985,927,396.0 |

## Non-stop (full burn)

Without stopping at the halfway point to slow down, journey time is much reduced. However you wouldn't see much at your destination, which would be passed at `peak c`.

You can reach the closest stars in about 2.5 years, and cross the galaxy in 12 years.

In [6]:
table.clear_rows()
for distance in distances:
    process_dist(distance, False)

markdown_table = table.get_string()
display(Markdown(markdown_table))

|    Light years | Prop Yrs |        Co'rd Yrs |                    Peak c |          Lorentz |      Apparent c |
|--------------: |--------: |----------------: |-------------------------: |----------------: |---------------: |
|              1 |     1.29 |              1.7 |                      0.87 |              2.0 |             0.7 |
|              2 |     1.72 |              2.8 |                     0.945 |              3.0 |             1.1 |
|              3 |     2.02 |              3.8 |                     0.969 |              4.0 |             1.4 |
|              4 |     2.24 |              4.8 |                     0.980 |              5.1 |             1.7 |
|              5 |     2.42 |              5.8 |                     0.986 |              6.1 |             2.0 |
|             10 |     3.02 |             10.9 |                    0.9960 |             11.3 |             3.3 |
|             50 |     4.51 |             50.9 |                   0.99981 |             52.6 |            11.0 |
|            100 |     5.17 |            100.9 |                  0.999953 |            104.2 |            19.3 |
|            200 |     5.83 |            200.9 |                  0.999988 |            207.4 |            34.2 |
|            500 |     6.72 |            500.9 |                 0.9999981 |            517.1 |            74.3 |
|          1,000 |     7.39 |          1,000.9 |                0.99999953 |          1,033.2 |           135.2 |
|          2,000 |     8.06 |          2,000.9 |                0.99999988 |          2,065.5 |           247.9 |
|          5,000 |     8.95 |          5,000.9 |               0.999999981 |          5,162.4 |           558.4 |
|         10,000 |     9.62 |         10,000.9 |              0.9999999953 |         10,323.9 |         1,039.0 |
|         20,000 |    10.29 |         20,000.9 |              0.9999999988 |         20,646.9 |         1,942.5 |
|         50,000 |    11.18 |         50,000.9 |             0.99999999981 |         51,615.7 |         4,470.8 |
|        100,000 |    11.85 |        100,000.9 |            0.999999999953 |        103,230.5 |         8,435.2 |
|        500,000 |    13.41 |        500,000.9 |           0.9999999999981 |        516,148.6 |        37,274.2 |
|      1,000,000 |    14.08 |      1,000,000.9 |          0.99999999999953 |      1,032,296.2 |        70,994.7 |
|      2,500,000 |    14.97 |      2,500,000.9 |         0.999999999999924 |      2,580,739.1 |       166,965.3 |
|      5,000,000 |    15.64 |      5,000,000.9 |         0.999999999999981 |      5,161,477.3 |       319,598.4 |
|     10,000,000 |    16.31 |     10,000,000.9 |        0.9999999999999953 |     10,322,953.7 |       612,891.7 |
|     25,000,000 |    17.20 |     25,000,000.9 |       0.99999999999999924 |     25,807,382.8 |     1,453,174.1 |
|  1,000,000,000 |    20.77 |  1,000,000,000.9 |    0.99999999999999999953 |  1,032,295,276.5 |    48,129,697.7 |
| 47,000,000,000 |    24.50 | 47,000,000,000.9 | 0.99999999999999999999978 | 48,517,877,952.0 | 1,917,828,112.8 |

## Other notebooks

Some other notebooks exploring special relativity are here:

https://github.com/lookbusy1344/Relativity/blob/main/Python
